Start



In [1]:
%load_ext autoreload
%autoreload 2

# Import some libraries

In [2]:

import os
import sys
import lmdb
import pickle
import numpy as np

# Cloning from github (Generative Models)



In [3]:
#!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach


# Cloning from github (LMDB_Datasets)

In [4]:
#!cd Generative_Models
#!git clone https://github.com/azeghost/LMDB_Datasets # this is for loading git with correct brach

In [5]:
import os
print(os.getcwd())

C:\Users\Kriple\PycharmProjects\Generative_Models\notebooks\Super_Resolution


In [6]:
sep_local = os.path.sep
#os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local)
os.chdir('C:/Users/Kriple/PycharmProjects/Generative_Models')
print(os.getcwd())

C:\Users\Kriple\PycharmProjects\Generative_Models


# LMDB transformer

In [7]:
#!pip -q install -r requirements.txt

In [8]:
from colorlog import ColoredFormatter

In [9]:
image_size=(300, 220, 3)

In [10]:
import tensorflow as tf

In [11]:
images_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68' #Folder to images ( without name of the folder which we moved images before)
validation_percentage = 20
valid_format = 'png'

# CBSD68_low2high_LMDB

In [12]:
inputs_shape = list(map(lambda x: x//2 , image_size[:-1])) + [image_size[-1]]
inputs_shape = (*inputs_shape,)

In [13]:
import cv2
import numpy as np

In [14]:
small_size = tuple([a//3 for a in image_size][:-1])
large_size = tuple([a for a in image_size][:-1])

In [15]:
shrink_fn = lambda image: np.asarray(cv2.resize(image, dsize=small_size, interpolation=cv2.INTER_CUBIC))
enlarg_fn = lambda image: np.asarray(cv2.resize(image, dsize=large_size, interpolation=cv2.INTER_CUBIC))

In [16]:
trans_fn = lambda image: enlarg_fn(shrink_fn(image))

In [17]:
from transformation.lmdb_transformer import SRLmdbTransformer

In [18]:
transformer = SRLmdbTransformer(image_dir=images_dir, trans_func=trans_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [19]:

val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

55 13


# Getting labels by filename

In [20]:
from transformation.transformative_custom_dataset import get_label_by_filename


In [21]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [22]:
lmdb_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB'
lmdb_dir

'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB'

In [23]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir,
           category='training', target_size=image_size[:-1], color_mode='rgb')


Total number of imgs for catagory 55
Storing 55 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB\ into _training from folder a


In [24]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir
           ,category='validation', target_size=image_size[:-1], color_mode='rgb')

Total number of imgs for catagory 13
Storing 13 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB\ into _validation from folder a


# Saving meta data of dataset

In [25]:
!ls C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB

_training
_validation


In [26]:

info = {
    'Dataset_name': "CBSD68_low2high",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata(lmdb_dir, info)

In [27]:
!ls -l C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB/

total 1
drwxr-xr-x 1 Kriple None   0 Nov  6 20:10 _training
drwxr-xr-x 1 Kriple None   0 Nov  6 20:10 _validation
-rw-r--r-- 1 Kriple None 111 Nov  6 20:10 meta_info.json


# CBSD68_denoising_LMDB

In [28]:
image_rev = tuple([*[*reversed(image_size)][1:], 3])

In [29]:
image_rev

(220, 300, 3)

In [30]:
noise_percentage=20/100
noise_image =  tf.random.normal(shape=image_rev)

trans_fn = lambda images: np.asarray(tf.clip_by_value((images + noise_image*noise_percentage),0,1).numpy())


In [31]:
from transformation.lmdb_transformer import SRLmdbTransformer

In [32]:
transformer = SRLmdbTransformer(image_dir=images_dir, trans_func=trans_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [33]:

val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

55 13


# Getting labels by filename

In [34]:
from transformation.transformative_custom_dataset import get_label_by_filename

In [35]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [36]:
lmdb_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_denoising_LMDB'
lmdb_dir

'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_denoising_LMDB'

In [37]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir,
           category='training', target_size=image_size[:-1], color_mode='rgb')


Total number of imgs for catagory 55
Storing 55 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_denoising_LMDB\ into _training from folder a


In [38]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir
           ,category='validation', target_size=image_size[:-1], color_mode='rgb')

Total number of imgs for catagory 13
Storing 13 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_denoising_LMDB\ into _validation from folder a


# Saving meta data of dataset

In [39]:
!ls C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_denoising_LMDB

_training
_validation


In [40]:

info = {
    'Dataset_name': "CBSD68_denoising_LMDB",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata(lmdb_dir, info)

In [41]:
!ls -l C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_denoising_LMDB/

total 1
drwxr-xr-x 1 Kriple None   0 Nov  6 20:10 _training
drwxr-xr-x 1 Kriple None   0 Nov  6 20:10 _validation
-rw-r--r-- 1 Kriple None 117 Nov  6 20:10 meta_info.json


# CBSD68_coloring_LMDB

In [42]:
image_rev = tuple([*[*reversed(image_size)][1:], 3])

In [43]:
image_rev

(220, 300, 3)

In [44]:
trans_fn = lambda images: np.asarray(tf.image.grayscale_to_rgb(tf.image.rgb_to_grayscale(images)).numpy())

In [45]:
from transformation.lmdb_transformer import SRLmdbTransformer

In [46]:
transformer = SRLmdbTransformer(image_dir=images_dir, trans_func=trans_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [47]:

val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

55 13


# Getting labels by filename

In [48]:
from transformation.transformative_custom_dataset import get_label_by_filename

In [49]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [50]:
lmdb_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_coloring_LMDB'
lmdb_dir

'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_coloring_LMDB'

In [51]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir,
           category='training', target_size=image_size[:-1], color_mode='rgb')


Total number of imgs for catagory 55
Storing 55 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_coloring_LMDB\ into _training from folder a


In [52]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir
           ,category='validation', target_size=image_size[:-1], color_mode='rgb')

Total number of imgs for catagory 13
Storing 13 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_coloring_LMDB\ into _validation from folder a


# Saving meta data of dataset

In [53]:
!ls C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_coloring_LMDB

_training
_validation


In [54]:

info = {
    'Dataset_name': "CBSD68_coloring_LMDB",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata(lmdb_dir, info)

In [55]:
!ls -l C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_coloring_LMDB/

total 1
drwxr-xr-x 1 Kriple None   0 Nov  6 20:11 _training
drwxr-xr-x 1 Kriple None   0 Nov  6 20:11 _validation
-rw-r--r-- 1 Kriple None 116 Nov  6 20:11 meta_info.json


# CBSD68_skitch_LMDB

In [56]:
image_rev = tuple([*[*reversed(image_size)][1:], 3])

In [57]:
image_rev

(220, 300, 3)

In [58]:
import random
import numpy as np
from utils.augmentation.augmentor import operations, do_operation
SKITCH=6
trans_fn = lambda images: np.asarray(do_operation(operations[SKITCH](), images, rotate=True))

In [59]:
from transformation.lmdb_transformer import SRLmdbTransformer

In [60]:
transformer = SRLmdbTransformer(image_dir=images_dir, trans_func=trans_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [61]:

val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

55 13


# Getting labels by filename

In [62]:
from transformation.transformative_custom_dataset import get_label_by_filename

In [63]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [64]:
lmdb_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_skitch_LMDB'
lmdb_dir

'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_skitch_LMDB'

In [65]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir,
           category='training', target_size=image_size[:-1], color_mode='rgb')


Total number of imgs for catagory 55
Storing 55 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_skitch_LMDB\ into _training from folder a


In [66]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir
           ,category='validation', target_size=image_size[:-1], color_mode='rgb')

Total number of imgs for catagory 13
Storing 13 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_skitch_LMDB\ into _validation from folder a


# Saving meta data of dataset

In [67]:
!ls C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_skitch_LMDB

_training
_validation


In [68]:

info = {
    'Dataset_name': "CBSD68_skitch_LMDB",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata(lmdb_dir, info)

In [69]:
!ls -l C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_skitch_LMDB/

total 1
drwxr-xr-x 1 Kriple None   0 Nov  6 20:11 _training
drwxr-xr-x 1 Kriple None   0 Nov  6 20:11 _validation
-rw-r--r-- 1 Kriple None 114 Nov  6 20:11 meta_info.json


# CBSD68_remove_text_LMDB

In [70]:
image_rev = tuple([*[*reversed(image_size)][1:], 3])

In [71]:
image_rev

(220, 300, 3)

In [72]:
import random
import string
characters = string.ascii_letters+string.ascii_letters+string.ascii_letters+string.digits+string.punctuation
word_len = range(10)
def random_sentance_generator():
    sentance = ''
    rn = random.choice(range(1, 20))
    for i in range(rn):
        sentance = sentance + ' ' + ''.join(random.choice(characters) for i in word_len)

    return sentance

In [73]:
from PIL import Image

In [74]:
fonts = list(set([eval('cv2.{}'.format(i)) for i in dir(cv2) if i.startswith('FONT_')]))
def put_random_text(image):

    image_dim = image.shape[:-1]
    #scale=1
    image_cv = cv2.cvtColor((image * 255).astype(np.uint8), cv2.IMREAD_COLOR)
    #image_cv = cv2.resize( image_cv, (image_dim[0]*scale, image_dim[1]*scale))
                          
    for _ in range(5):
        x_loc = random.choice(range(image_cv.shape[0]))
        y_loc = random.choice(range(image_cv.shape[1]))
        loc = (x_loc, y_loc)


        cv2.putText(img=image_cv, 
                    text=random_sentance_generator(), 
                    org=loc,
                    fontFace=random.choice(fonts),
                    fontScale=random.choice(range(1, 3)),
                    color=(random.choice(range(256)), random.choice(range(256)), random.choice(range(256))),
                    thickness=random.choice(range(3))
                   )
                
    image_cv = np.asarray(Image.fromarray(np.asarray(cv2.resize(image_cv, image_dim)).astype(np.uint8), mode='RGB')\
                          .rotate(90, expand=True))/255.0

    return image_cv

In [75]:
def put_text_fn(images):
    if len(images.shape)<4:
        images = [images]
    return np.array([put_random_text(image) for image in images])

In [76]:
trans_fn = lambda images: np.asarray(put_text_fn(images))

In [77]:
from transformation.lmdb_transformer import SRLmdbTransformer

In [78]:
transformer = SRLmdbTransformer(image_dir=images_dir, trans_func=trans_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [79]:
val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

55 13


# Getting labels by filename

In [80]:
from transformation.transformative_custom_dataset import get_label_by_filename

In [81]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [82]:
lmdb_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_remove_text_LMDB'
lmdb_dir

'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_remove_text_LMDB'

In [83]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir,
           category='training', target_size=image_size[:-1], color_mode='rgb')


Total number of imgs for catagory 55
Storing 55 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_remove_text_LMDB\ into _training from folder a


In [84]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir
           ,category='validation', target_size=image_size[:-1], color_mode='rgb')

Total number of imgs for catagory 13
Storing 13 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_remove_text_LMDB\ into _validation from folder a


# Saving meta data of dataset

In [85]:
!ls C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_remove_text_LMDB

_training
_validation


In [86]:

info = {
    'Dataset_name': "CBSD68_skitch_LMDB",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata(lmdb_dir, info)

In [87]:
!ls -l C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_remove_text_LMDB/

total 1
drwxr-xr-x 1 Kriple None   0 Nov  6 20:11 _training
drwxr-xr-x 1 Kriple None   0 Nov  6 20:11 _validation
-rw-r--r-- 1 Kriple None 114 Nov  6 20:11 meta_info.json
